# Tangram

 We followed the procedure outlined in the Tangram GitHub repository: https://github.com/broadinstitute/Tangram. Tangram is an integration method to combine spatial transcriptomics and single-cell RNA-seq data using a non-convex optimization alongside deep learning to leart a spatial alignment for single cell data.

# Imports

In [1]:
global_path = './cell-cell-communication/' ## Path to the github downloaded repository

In [2]:
import scanpy as sc
import tangram as tg
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80, facecolor="white")

/home/epareja/miniconda3/envs/tangram-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load the data

In [5]:
## NB: in this notebook we use the SAN region. For the AVN is the same, just change the files. 
# scRNA-seq data of the region and the patient
adata_sc = sc.read(global_path + 'Data/Heart/Global_lognormalised_SAN_AH1.h5ad')
# Spatial data of the region and select the patient
adata_st = sc.read(global_path + 'Data/Heart/visium-OCT_SAN_lognormalised.h5ad')
adata_st = adata_sp[adata_sp.obs.donor == 'AH1']

## Select common genes between spatial and the reference dataset

In [6]:
markers = list(set.intersection(set(adata_sc.var_names), set(adata_st.var_names)))
tg.pp_adatas(adata_sc, adata_st, genes=markers)


filtered out 3251 genes that are detected in less than 1 cells
filtered out 12278 genes that are detected in less than 1 cells


INFO:root:20399 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:20399 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.


In [8]:
## check if the genes are in the adata
assert "training_genes" in adata_sc.uns
assert "training_genes" in adata_st.uns

## Map cells to spots

In [9]:
ad_map = tg.map_cells_to_space(
    adata_sc,
    adata_st,
    mode="cells",
    density_prior="rna_count_based",
    num_epochs=500,
    device="cuda",  # or: cpu
)

INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 20399 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.203, KL reg: 0.013
Score: 0.414, KL reg: 0.000
Score: 0.426, KL reg: 0.000
Score: 0.430, KL reg: 0.000
Score: 0.431, KL reg: 0.000


INFO:root:Saving results..
INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [10]:
## project cell annotations (in out case cell states) in the space: this is for visualization
tg.project_cell_annotations(ad_map, adata_st, annotation="cell_state")

INFO:root:spatial prediction dataframe is saved in `obsm` `tangram_ct_pred` of the spatial AnnData.


## Select only one best spot for each cell

For each cell, select the spot with the highest probability

In [11]:
# Select the information of the spots
annotation_list = list(pd.unique(adata_sc.obs["cell_state"]))
coords = ad_map.var[['array_row', 'array_col']]
annotation_spots = np.concatenate(ad_map.var[['annotation_final']].values)
spots_order = ad_map.var.index.values


In [12]:
# Add for each cell the most probable spot with its metadata
argsort_prob = ad_map.X.argsort()
most_prob_spot = [argsort_prob[n][-1] for n in range(ad_map.X.shape[0])]

cell_coords_x = [coords.values[l][0] for l in most_prob_spot]
cell_coords_y = [coords.values[l][1] for l in most_prob_spot]
cell_annotation = [annotation_spots[l] for l in most_prob_spot]
cell_spots = [spots_order[l] for l in most_prob_spot]

ad_map.obs['cell_coords_x'] = cell_coords_x
ad_map.obs['cell_coords_y'] = cell_coords_y
ad_map.obs['cell_annotation'] = cell_annotation
ad_map.obs['cell_spots'] = cell_spots

In [16]:
ad_map.write_h5ad(global_path + 'Results/Heart/Tangram/SAN_ad_map.h5ad')